In [1]:
#Libraries
import numpy as np
import natsort
import cv2 as cv
import matplotlib.pyplot as plt
import copy as copy
import math
import os
import functions as ftn
import imageio
import scipy.ndimage
import disparity_functions as disp

PARAMETERS

In [2]:
FOV = 110 # degree
height = 640
width = 640
K = np.array([[2.240664122271071506e+02, 0.000000000000000000e+00, 3.200000000000000000e+02],
            [0.000000000000000000e+00, 2.240664122271071506e+02, 3.200000000000000000e+02],
            [0.000000000000000000e+00, 0.000000000000000000e+00, 1.000000000000000000e+00]])

f_x = K[0,0]
f_y = K[1,1]
r = K[0,2]
s = K[1,2]
B = 0.3
K_inv = np.linalg.inv(K)

# Parameters
num_disp_values=255
Lambda=10.0
nb_iterations=15

1. LOAD IMAGES

In [3]:
# recall color and gray images and mask

path = './StereoImages/'
order_list = os.listdir(path)
after_order_list = natsort.natsorted(order_list)
print(after_order_list)
# left and right color images 
colors_left_list = []
colors_right_list = []

# left and right gray images
grays_left_list = []
grays_right_list = []

for root, directions, files in os.walk(path):
    
    for file in after_order_list:
        img = cv.imread(os.path.join(root, file))
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        
        if 'ref.png' in file:  
            colors_left_list.append(img)
            grays_left_list.append(gray)
        
        if 'tar.png' in file:
            colors_right_list.append(img)
            grays_right_list.append(gray)


['00000_ref.png', '00000_tar.png', '00001_ref.png', '00001_tar.png', '00002_ref.png', '00002_tar.png', '00003_ref.png', '00003_tar.png', '00004_ref.png', '00004_tar.png', '00005_ref.png', '00005_tar.png', '00006_ref.png', '00006_tar.png', '00007_ref.png', '00007_tar.png', '00008_ref.png', '00008_tar.png', '00009_ref.png', '00009_tar.png', '00010_ref.png', '00010_tar.png', '00011_ref.png', '00011_tar.png', '00012_ref.png', '00012_tar.png', '00013_ref.png', '00013_tar.png', '00014_ref.png', '00014_tar.png', '00015_ref.png', '00015_tar.png', '00016_ref.png', '00016_tar.png', '00017_ref.png', '00017_tar.png', '00018_ref.png', '00018_tar.png', '00019_ref.png', '00019_tar.png', '00020_ref.png', '00020_tar.png']


2. FEATURE DETECTION

In [4]:
raw_key_pts_list = [] # raw key point list for each left image
descriptors_list = [] # descriptor list for each left image

#detector = cv.BRISK_create()
detector = cv.ORB_create()
for i in range(21):
    
    raw_key_pts, descriptors = detector.detectAndCompute(grays_left_list[i], None)
    raw_key_pts = np.float32([key_pt.pt for key_pt in raw_key_pts])
    raw_key_pts_list.append(raw_key_pts)
    descriptors_list.append(descriptors)


3. FIND DIFFERENCE ANGLE BETWEEN A PAIR OF IMAGES 

In [5]:
rot_angle_list = []
sum = 0
for i in range(20):
    # a. find right features
    key_pts1, key_pts2, matches = ftn.findRightFeatures(raw_key_pts_list[i], raw_key_pts_list[i+1], descriptors_list[i], descriptors_list[i+1])
    
    #print('%d, %d' % (len(key_pts1), len(key_pts2)))
    key_pts1 = ftn.homoPixel2Camera(key_pts1, K_inv)
    key_pts2 = ftn.homoPixel2Camera(key_pts2, K_inv)
    
    key_pts1_cylinder = ftn.transformHomo2Cylinder(key_pts1)
    key_pts2_cylinder = ftn.transformHomo2Cylinder(key_pts2)
    # b. find homogeneous coord in real space    
    
    theta = np.degrees(np.mean(key_pts2_cylinder[:,0] - key_pts1_cylinder[:,0]))
    rot_angle_list.append(theta)

print(np.sum(np.float32(rot_angle_list)))
rot_angle_list.insert(0,0)

rot_angle_list = 360/np.sum(rot_angle_list)*np.float32(rot_angle_list)

    


359.92795


4. OMNIDIRECTIONAL IMAGE RESTORATION

In [6]:
colors_cylinder = []
factor = 4
width = factor*470
height = 640
angle = 0
# generate mask
mask = 255 * np.uint8(np.ones((640, 640)))
mask = ftn.cylindricalWarp(mask, K)
mask = cv.cvtColor(mask, cv.COLOR_BGR2GRAY)
ret, mask = cv.threshold(mask, 254, 255, cv.THRESH_BINARY)
min_idx_width, max_idx_width = ftn.findImgWidthDim(mask)
mask = mask[:, min_idx_width:max_idx_width+1]
mask = cv.resize(mask, (factor*110, height))
mask_inv = cv.bitwise_not(mask)



#combined image
image_combined = np.uint8(np.zeros((height, width, 3)))

# omnidirectional image restoration
for i in range(21):
    
    # one omni. image
    img_cyl = ftn.cylindricalWarp(colors_left_list[i], K)
    img_cyl = img_cyl[:, :,0:3]
    img_cyl = img_cyl[:, min_idx_width:max_idx_width+1]
    img_cyl= cv.resize(img_cyl, (factor*110, height))
    
    # pixel movement
    angle = angle + rot_angle_list[i]
    angle_move = round(factor*angle)
    
    # restoration
    roi = image_combined[:, width-angle_move-factor*110:width-angle_move]
    
    img_cyl_bg = cv.bitwise_and(roi, roi, mask=mask_inv)
    img_cyl_fg = cv.bitwise_and(img_cyl, img_cyl, mask=mask)
    dst = cv.add(img_cyl_bg, img_cyl_fg)
    
    image_combined[:, width-factor*110-angle_move:width-angle_move] = dst

image_combined = image_combined[:, 55*factor:width-55*factor]
_, w, _ = image_combined.shape
dst =  copy.deepcopy(image_combined[:,0:round(w/2)])
image_combined[:,0:round(w/2)] = image_combined[:,round(w/2):]
image_combined[:,round(w/2):] = dst
cv.imwrite('./combined.png', image_combined)



True

5. OMNIDIRECTIONAL IMAGE RESTORATION
6. OMNIDIRECTIONAL DISTANCE RESTORATION

In [20]:
# generate disparity map using Loopy Belief Propagation
disp_list = []

for i in range(21):

    img_left =colors_left_list[i]
    img_right=colors_right_list[i]

    # Convert as float gray images
    img_left=img_left.astype(float)
    img_right=img_right.astype(float)

    # Gaussian filtering
    I1=scipy.ndimage.filters.gaussian_filter(img_left, 0.6)
    I2=scipy.ndimage.filters.gaussian_filter(img_right, 0.6)

    disparity,energy = disp.stereo_bp(I1,I2,num_disp_values,Lambda, num_iterations=nb_iterations)
    
    disparity.astype(float)

    h, w = disparity.shape
    for j in range(h):
        for k in range(w):
            disparity[j, k] = min(f_x * B / (disparity[j, k] + 1e-8), 30) / 30 * 65535
    disparity = np.array(disparity, dtype=np.uint16)
    disparity=scipy.ndimage.filters.gaussian_filter(disparity, 0.6)

    cv.imwrite('./disparity/disparity_%d.png'%i,disparity)

    


Iteration (out of 15) :
	0
	7
	14


In [25]:
disp_list = []
# post-processing
for i in range(21):
    path = './disparity/disparity_%d.png'%i
    disparity = cv.imread(path, -1)
    for j in range(640):
        for k in range(640):
            if disparity[j, k] > 60000:
                disparity[j, k] = 25000
    disp_list.append(disparity)
            


In [27]:
factor = 4
width = factor*470
height = 640
angle = 0
disp_combined = np.uint16(np.zeros((height, width)))


for i in range(21):
    
    disp_cyl = ftn.cylindricalWarp_disp(disp_list[i], K)
    #disp_cyl = disp_cyl[:, :,0:3]
    disp_cyl = disp_cyl[:, min_idx_width:max_idx_width+1]
    disp_cyl= cv.resize(disp_cyl, (factor*110, height))
    
    # pixel movement
    angle = angle + rot_angle_list[i]
    angle_move = round(factor*angle)
    
    # restoration
    roi = disp_combined[:, width-angle_move-factor*110:width-angle_move]
    
    disp_cyl_bg = cv.bitwise_and(roi, roi, mask=mask_inv)
    disp_cyl_fg = cv.bitwise_and(disp_cyl, disp_cyl, mask=mask)
    dst = cv.add(disp_cyl_bg, disp_cyl_fg)
    disp_combined[:, width-factor*110-angle_move:width-angle_move] = dst

disp_combined = disp_combined[:, 55*factor:width-55*factor]
_, w = disp_combined.shape
dst =  copy.deepcopy(disp_combined[:,0:round(w/2)])
disp_combined[:,0:round(w/2)] = disp_combined[:,round(w/2):]
disp_combined[:,round(w/2):] = dst
cv.imwrite('./disp_combined.png', disp_combined)
print(disp_combined.dtype)

uint16


uint16


7. SAVE ROTATION ANGLE

In [ ]:
rot_angle_list = rot_angle_list[1:]
f = open('./rotation_angle.txt', 'w')
for angle in rot_angle_list:
    f.write('%f ' % angle)
f.close()